In [ ]:
from icf.frame import Frame
import numpy as np
from datetime import datetime
from icf.pyicf import ICFFile

import pandas as pd

In [ ]:
from icf.frame import dispatch_serializer
import struct
import numpy as np
from importlib import import_module
import logging
import sys
class TopicFrame(Frame):
    
    def serialize(self) -> bytes:
        """Serializes the frame to a bytestream

        Returns:
            bytes: serialized frame
        """
        data_stream = bytearray()
        index = []
        classes = []
        pos = 0
        for k, v in self.items():
            v = dispatch_serializer(v)
            _module = v.__class__.__module__
            _module = _module if _module not in [__name__,"icf.frame"] else ""
            classes.append("{},{},{}\n".format(k, v.__class__.__name__, _module))
            d = v.serialize()
            pos += len(d)
            index.append(pos)
            data_stream.extend(d)
        n_obj = len(index)

        classes = "".join(classes)
        header = bytearray(struct.pack(
            "<2H{}I{}s".format(n_obj, len(classes)),
            len(classes),
            n_obj,
            *index,
            classes.encode(),
        ))
        print(classes)
        header.extend(data_stream)
        return header
    
    @classmethod
    def deserialize_header(cls,data):
        ncls, nobj=struct.unpack("<2H",data[:4])
        print(ncls,nobj,ncls+4*nobj)
        index = struct.unpack("<{}I{}s".format(nobj, ncls), data[4:ncls+4*nobj+4])
        classes = index[nobj:][0].decode()
        index = list(index[:nobj])
        print(classes)
        print(index)
        
class FileStream:
    def __init__(self, path, mode=""):
        self.buffer = bytearray()
        self.size = len(self.buffer)
        self.fp = 0
        self.path = path
        self.mode = mode
        
    def read(self, size=-1):
        if size == -1:
            size = self.size
        self.fp += size
        return self.buffer[self.fp-size:self.fp]
    
    def write(self, buffer):
        self.buffer.extend(buffer)
        self.size = len(self.buffer)
        self.fp = self.size
        return len(buffer)
    def flush(self):
        pass
    def close(self):
        pass
    def tell(self):
        return self.fp
    
    def seek(self,cookie, whence=0):
        if whence == 0:
            self.fp = cookie
            return self.fp
        if whence == 1:
            self.fp += cookie
            return self.fp
        if whence == 2:
            self.fp = self.size+cookie
            return self.fp

In [ ]:
frame = TopicFrame()
frame.add("First",1)
frame.add("Second", np.array([123,2,3,54,5,3,23,4,35,42,3,43,],dtype=np.uint8))
print(frame.serialize())
print(len(frame.serialize()))
TopicFrame.deserialize_header(frame.serialize())
frame = Frame()
frame.add("First",1)
frame.add("Second", [123,2,3,54,5,3,23,4,35,42,3,43,])
print(frame.serialize())
print(len(frame.serialize()))
f = FileStream(frame.serialize())

In [ ]:
file = ICFFile("test",custom_stream=FileStream("test"))
file.write(frame.serialize())
frame["some array"] = np.arange(10)
file.write(frame.serialize())
file.flush()
file.close()
file._file.buffer


In [ ]:
filer= ICFFile("test.icf","trunc")

filer.write(frame.serialize())

filer.write(frame.serialize())
filer.flush()
#file._file.buffer
filer.close()
d =  open("test.icf","rb").read()

In [ ]:
print(len(d))
print(len(file._file.buffer))
print(d)

In [ ]:
Frame.deserialize(file.read_at(1))

In [ ]:
end = file._file.seek(0,2)
file._file.seek(end-4)
d = file._file.read(4)
i = struct.unpack("<I", d)[0]
print(d,i)

In [ ]:
file._file.buffer.extend(file._file.buffer)
file._file.size = len(file._file.buffer)

In [ ]:
print(file._file.buffer)

file._file.seek(0)
file2 = ICFFile("test",custom_stream=file._file)
print(file2)
Frame.deserialize(file2.read_at(3))['some array']

In [ ]:
print(f.read(10))
print(f.read(10))
print(f.tell())
print(f.seek(0))
print(f.read(10))
print(f.seek(-8,2))

In [ ]:

startTime = datetime.now()
writer = ICFFile("testing.icf", compressor=None)  #'bz2')

for i in range(90):

    frame = Frame()
    frame.add("r",1)
    frame.add("r2", np.random.uniform(0,1012321))
    #frame["a_list_of_lists"] = [1, 3, 4, 5, [9, 4, 5], (93, 3.034)]
    writer.write(frame.serialize())
writer.close()